# 步骤 1: 准备工作与环境设置

**目标:** 导入所有需要的库，并设置好文件路径和全局变量。

In [1]:
# --- 步骤 1: 准备工作与环境设置 (已集成5级评分) ---

# --- 核心库 ---
import html
import json
import os
import re
import time
import unicodedata
from itertools import islice # 为测试模式引入

# --- 数据处理与NLP库 ---
import pandas as pd
import psutil
import spacy
from flashtext import KeywordProcessor
from tqdm.auto import tqdm

# --- 快速测试模式开关 ---
TEST_MODE = False
TEST_SAMPLE_SIZE = 50000
CANDIDATE_SAMPLE_SIZE = 2000

# --- 全局文件与参数配置 ---
ALIYUN_OSS_PATH = ''
KEYWORD_JSON_PATH = os.path.join(ALIYUN_OSS_PATH, '../data_raw/china_keywords_collection.json')
SOURCE_NEWS_FILE = os.path.join(ALIYUN_OSS_PATH, '../data_raw/final_merged_all_news.csv')
CANDIDATES_FILE = os.path.join(ALIYUN_OSS_PATH, '../data_processed/china_news_candidates.csv')
FINAL_RESULT_FILE = os.path.join(ALIYUN_OSS_PATH, '../data_processed/final_china_news.csv')
REJECTED_FILE = os.path.join(ALIYUN_OSS_PATH, '../data_processed/china_news_rejected_articles.csv')

# --- 全局处理参数 ---
NEWS_COLUMN = 'CONTENT'
CHUNKSIZE = 20000

# --- 智能配置并行参数 ---
cpu_cores = psutil.cpu_count(logical=False)
N_PROCESSES = min(cpu_cores - 1 if cpu_cores > 1 else 1, 8)
if N_PROCESSES < 1: N_PROCESSES = 1
BATCH_SIZE = 500

# --- 精筛积分系统配置 (已升级为5级) ---
ACCEPTANCE_THRESHOLD = 5.0      # 总分高于此值则接受

# -- 正向加分 --
# 前10句动态奖励
LEAD_BONUS_TIER_5 = 20.0
LEAD_BONUS_TIER_4 = 15.0
LEAD_BONUS_TIER_3 = 10.0
LEAD_BONUS_TIER_2 = 5.0
# 关键词频率分
TIER_5_SCORE = 5.0
TIER_4_SCORE = 4.0
TIER_3_SCORE = 3.0
TIER_2_SCORE = 2.0
TIER_1_SCORE = 1.0
# -- 负向扣分 --
NEGATION_PENALTY = -3.0
HYPOTHETICAL_PENALTY = -2.0

print("✅ 块 1: 库导入和配置完成。")
print("-" * 30)
if TEST_MODE:
    print(f"🚀🚀🚀 运行在【快速测试模式】下！🚀🚀🚀")
else:
    print("🚢🚢🚢 运行在【完整数据模式】下。🚢🚢🚢")
print("   - 精筛将采用升级版5级积分制，接受阈值为: " f"{ACCEPTANCE_THRESHOLD} 分")
print("-" * 30)
print(f"   - 初筛将使用单进程顺序处理。")
print(f"   - 精筛阶段将使用 {N_PROCESSES} 个进程进行并行处理。")

✅ 块 1: 库导入和配置完成。
------------------------------
🚢🚢🚢 运行在【完整数据模式】下。🚢🚢🚢
   - 精筛将采用升级版5级积分制，接受阈值为: 5.0 分
------------------------------
   - 初筛将使用单进程顺序处理。
   - 精筛阶段将使用 5 个进程进行并行处理。


# 步骤 2: 初筛准备 - 构建智能正则表达式

**目标:** 读取 中国相关关键词 JSON 文件，并执行我们讨论过的所有逻辑来构建一个强大、高效的正则表达式。

In [2]:
# --- 构建初筛用的 Flashtext 关键词处理器 ---

def build_keyword_processor(json_path):
    """
    从关键词 JSON 文件中构建一个高效的 Flashtext KeywordProcessor。
    """
    print(f"正在从 {json_path} 加载关键词...")
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            keywords_data = json.load(f)
    except FileNotFoundError:
        print(f"❌ 错误: 关键词文件未找到 {json_path}")
        return None, None

    print(f"共加载 {len(keywords_data)} 个关键词对象。")

    # 1. 提取全部关键词和别名
    all_aliases = set()
    for item in keywords_data:
        all_aliases.add(item['keyword'])
        for alias in item.get('aliases', []):
            all_aliases.add(alias)
    print(f"提取出 {len(all_aliases)} 个不重复的关键词/别名。")

    # 2. 初始化 Flashtext 处理器并添加关键词
    # case_sensitive=False 使其不区分大小写
    keyword_processor = KeywordProcessor(case_sensitive=False)
    for kw in all_aliases:
        keyword_processor.add_keyword(kw)

    print("✅ 高效关键词处理器 (Flashtext) 构建完成。")
    return keyword_processor, keywords_data


# 执行构建
keyword_processor, keywords_data = build_keyword_processor(KEYWORD_JSON_PATH)

# 将关键词处理器设为全局变量，以便后续子进程可以访问 (在某些系统上需要)
global_keyword_processor = keyword_processor

print("\n✅ 块 2: 初筛准备工作完成。")

正在从 ../data_raw/china_keywords_collection.json 加载关键词...
共加载 274 个关键词对象。
提取出 394 个不重复的关键词/别名。
✅ 高效关键词处理器 (Flashtext) 构建完成。

✅ 块 2: 初筛准备工作完成。


# 步骤 3: 执行阶段一 - 调用外部脚本进行快速初筛

**目标:** 对大文件进行分块扫描，应用正则表达式，并保存候选集。这将是整个流程中最耗时的部分。

In [3]:
# --- 步骤 3: 执行第一阶段 - 大规模流式初筛 (已优化进度条) ---

# --- 定义处理函数 ---
def lightweight_clean(text):
    if not isinstance(text, str): return ""
    text = re.sub('<[^>]*>', '', text)
    text = html.unescape(text)
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def process_chunk(chunk_df):
    if NEWS_COLUMN not in chunk_df.columns: return pd.DataFrame()
    cleaned_series = chunk_df[NEWS_COLUMN].astype(str).apply(lightweight_clean)
    mask = cleaned_series.apply(lambda x: len(keyword_processor.extract_keywords(x)) > 0)
    return chunk_df[mask]

# --- 执行阶段一 - 流式扫描与初筛 (已优化进度条) ---
print("--- 阶段一: 开始使用单进程进行流式初筛 ---")
start_time = time.time()

try:
    # [关键变更] 使用更可靠的方式计算总块数
    if not TEST_MODE:
        print("正在计算文件总块数 (优化方式)...")
        # 使用 pandas 快速迭代来获取准确的行数，这会考虑坏行
        # 我们只读取第一列来最小化内存占用和加快速度
        try:
            first_col_name = pd.read_csv(SOURCE_NEWS_FILE, nrows=0).columns[0]
            row_iterator = pd.read_csv(SOURCE_NEWS_FILE, chunksize=100000, usecols=[first_col_name], on_bad_lines='skip', low_memory=False)
            num_lines = sum(len(chunk) for chunk in row_iterator)
            total_chunks = (num_lines // CHUNKSIZE) + 1
            print(f"文件包含 {num_lines} 有效行, 约 {total_chunks} 个数据块。")
        except Exception as e:
            print(f"快速计算行数失败: {e}. 将不显示总进度。")
            total_chunks = None # 如果计算失败，则不显示总进度

    # 创建用于实际处理的迭代器
    chunk_iterator = pd.read_csv(SOURCE_NEWS_FILE, chunksize=CHUNKSIZE, on_bad_lines='skip', low_memory=False)

    if TEST_MODE:
        num_test_chunks = (TEST_SAMPLE_SIZE // CHUNKSIZE) + 1
        chunk_iterator = islice(chunk_iterator, num_test_chunks)
        total_chunks = num_test_chunks
        print(f"🚀 测试模式: 将处理前 {total_chunks} 个数据块 (约 {TEST_SAMPLE_SIZE} 行)。")

    print("将使用单进程顺序处理...")

    is_first_chunk = True
    total_candidates = 0

    # --- 使用 tqdm 包装迭代器 ---
    # 如果 total_chunks 为 None (计算失败时)，tqdm 会自动回退到不显示百分比的模式
    for chunk_df in tqdm(chunk_iterator, total=total_chunks, desc="顺序初筛中"):
        candidates_df = process_chunk(chunk_df)

        if not candidates_df.empty:
            total_candidates += len(candidates_df)
            if is_first_chunk:
                candidates_df.to_csv(CANDIDATES_FILE, index=False, mode='w', encoding='utf-8')
                is_first_chunk = False
            else:
                candidates_df.to_csv(CANDIDATES_FILE, index=False, mode='a', header=False, encoding='utf-8')

    end_time_stage1 = time.time()
    print("\n--- 初筛流程执行完毕 ---")
    print(f"总共找到 {total_candidates} 篇候选文章，已保存到 {CANDIDATES_FILE}")
    print(f"阶段一 (顺序初筛) 耗时: {(end_time_stage1 - start_time):.2f} 秒。")

except FileNotFoundError:
    print(f"❌ 错误: 原始新闻文件未找到 {SOURCE_NEWS_FILE}")
except Exception as e:
    print(f"❌ 处理过程中发生错误: {e}")

print("\n✅ 块 3: 初筛流程执行完毕。")

--- 阶段一: 开始使用单进程进行流式初筛 ---
正在计算文件总块数...
文件约包含 189 个数据块。
将使用单进程顺序处理...


顺序初筛中:   0%|          | 0/189 [00:00<?, ?it/s]


--- 初筛流程执行完毕 ---
总共找到 179380 篇候选文章，已保存到 ../data_processed/china_news_candidates.csv
阶段一 (顺序初筛) 耗时: 837.96 秒。

✅ 块 3: 初筛流程执行完毕。


# 步骤 4: 精筛准备 - 加载模型与定义规则

**目标:** 负责加载 spaCy 模型和数据，并定义所有用于精筛的“否决规则”函数。

In [4]:
# --- 步骤 4: 准备第二阶段 - 构建多维度相关性评估引擎 (已升级为5级评分) ---
print("--- 阶段二准备: 加载 spaCy 及构建评估规则 ---")

try:
    nlp = spacy.load("en_core_web_lg", disable=["ner", "lemmatizer", "tagger", "attribute_ruler"])
    print(f"✅ spaCy 模型 '{nlp.meta['name']}' 的核心组件加载成功。")
except OSError:
    print("错误: spaCy模型 'en_core_web_lg' 未安装。请运行: python -m spacy download en_core_web_lg")
    nlp = None

if nlp:
    # 1. 构建关键词信息查找表
    print("正在构建关键词信息查找表...")
    keyword_lookup = {}
    for item in keywords_data:
        tier = item.get('relevance_tier', 1)
        keyword_lookup[item['keyword'].lower()] = {'tier': tier}
        for alias in item.get('aliases', []):
            keyword_lookup[alias.lower()] = {'tier': tier}
    print(f"✅ 查找表构建完成，包含 {len(keyword_lookup)} 个词条。")

    # 2. 构建 PhraseMatcher
    print("正在准备 PhraseMatcher...")
    from spacy.matcher import PhraseMatcher
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = [nlp.make_doc(text) for text in keyword_lookup.keys()]
    matcher.add("ChinaKeywords", patterns)
    print(f"✅ PhraseMatcher 准备完成，已添加 {len(patterns)} 个模式。")

    # 3. 定义积分/扣分规则函数
    # [已升级] score_keyword_frequency 现在支持5个等级
    def score_keyword_frequency(matches, doc, lookup):
        score = 0
        for match_id, start, end in matches:
            kw = doc[start:end].text.lower()
            tier = lookup.get(kw, {}).get('tier', 1)
            if tier == 5: score += TIER_5_SCORE
            elif tier == 4: score += TIER_4_SCORE
            elif tier == 3: score += TIER_3_SCORE
            elif tier == 2: score += TIER_2_SCORE
            else: score += TIER_1_SCORE # tier 1 or unknown
        return score

    # [已升级] score_lead_paragraphs_presence 现在是动态奖励
    def score_lead_paragraphs_presence(doc, matcher, lookup):
        sents = list(doc.sents)
        if not sents: return 0, ""

        lead_sents_count = min(len(sents), 10)     # 前10个句子
        lead_end_token_index = sents[lead_sents_count - 1].end

        matches = matcher(doc)

        highest_tier_in_lead = 0
        found_kw = ""

        for match_id, start, end in matches:
            if start < lead_end_token_index:
                kw = doc[start:end].text.lower()
                tier = lookup.get(kw, {}).get('tier', 1)
                if tier > highest_tier_in_lead:
                    highest_tier_in_lead = tier
                    found_kw = kw

        if highest_tier_in_lead == 5: return LEAD_BONUS_TIER_5, f"前两段加分-T5(+'{found_kw}')"
        if highest_tier_in_lead == 4: return LEAD_BONUS_TIER_4, f"前两段加分-T4(+'{found_kw}')"
        if highest_tier_in_lead == 3: return LEAD_BONUS_TIER_3, f"前两段加分-T3(+'{found_kw}')"
        if highest_tier_in_lead == 2: return LEAD_BONUS_TIER_2, f"前两段加分-T2(+'{found_kw}')"

        return 0, ""

    def penalize_hypothetical(doc, keywords_in_doc):
        penalty = 0
        reasons = []
        for sent in doc.sents:
            clean_sent_start = sent.text.strip().lower()
            if clean_sent_start.startswith(('if ', 'unless ', 'what if')):
                if any(token.text.lower() in keywords_in_doc for token in sent):
                    penalty += HYPOTHETICAL_PENALTY
                    reasons.append(f"假设句扣分: '{sent.text[:50].strip()}...'")
        return penalty, reasons

    def penalize_negation(doc, keywords_in_doc):
        penalty = 0
        reasons = []
        for token in doc:
            if token.dep_ == "neg" and token.head.text.lower() in keywords_in_doc:
                penalty += NEGATION_PENALTY
                reasons.append(f"否定扣分: '{token.text} {token.head.text}'")
        return penalty, reasons

    print("✅ 块 4: 精筛规则和评估引擎准备完成 (已升级为5级评分)。")

--- 阶段二准备: 加载 spaCy 及构建评估规则 ---
✅ spaCy 模型 'core_web_lg' 的核心组件加载成功。
正在构建关键词信息查找表...
✅ 查找表构建完成，包含 392 个词条。
正在准备 PhraseMatcher...
✅ PhraseMatcher 准备完成，已添加 392 个模式。
✅ 块 4: 精筛规则和评估引擎准备完成 (已升级为5级评分)。


# 步骤 5: 执行阶段二 - 精筛流程

**目标:** 加载候选集，应用所有否决规则，然后保存最终结果和被拒绝的文章。

In [5]:
# --- 步骤 5: 执行第二阶段 - 上下文精筛与产出 (最终版) ---
print("--- 阶段二: 开始精筛候选集 ---")
start_time_s2 = time.time()

try:
    read_csv_kwargs = {'low_memory': False}
    if TEST_MODE:
        read_csv_kwargs['nrows'] = CANDIDATE_SAMPLE_SIZE
        print(f"🚀 测试模式: 最多加载前 {CANDIDATE_SAMPLE_SIZE} 篇候选文章进行精筛。")

    df_candidates = pd.read_csv(CANDIDATES_FILE, **read_csv_kwargs)
    print(f"✅ 成功加载 {len(df_candidates)} 篇候选文章。")
except FileNotFoundError:
    print(f"❌ 错误: 候选文件未找到 {CANDIDATES_FILE}。请先运行块 3。")
    df_candidates = pd.DataFrame()

if not df_candidates.empty and nlp:
    texts = df_candidates[NEWS_COLUMN].astype(str)
    results = []

    print(f"开始使用 {N_PROCESSES} 个进程并行处理文本...")
    docs_content = nlp.pipe(texts, batch_size=BATCH_SIZE, n_process=N_PROCESSES)

    for doc in tqdm(docs_content, total=len(df_candidates), desc="精筛文章"):
        score = 0
        score_details = []

        matches = matcher(doc)
        if not matches:
            results.append({'score': 0, 'reason': '精筛阶段未匹配到任何关键词'})
            continue

        found_keywords_text = {doc[start:end].text.lower() for _, start, end in matches}

        # 2. 正向加分
        lead_score, lead_reason = score_lead_paragraphs_presence(doc, matcher, keyword_lookup)
        if lead_score > 0: score_details.append(lead_reason)

        freq_score = score_keyword_frequency(matches, doc, keyword_lookup)
        if freq_score > 0: score_details.append(f"关键词频率分: +{freq_score:.2f}")

        score = lead_score + freq_score

        # 3. 负向扣分
        hypo_penalty, hypo_reasons = penalize_hypothetical(doc, found_keywords_text)
        nega_penalty, nega_reasons = penalize_negation(doc, found_keywords_text)

        score += hypo_penalty
        score += nega_penalty
        score_details.extend(hypo_reasons)
        score_details.extend(nega_reasons)

        results.append({'score': score, 'reason': ' | '.join(score_details)})

    # 4. 合并与保存结果
    print("\n正在合并精筛结果...")
    df_results = pd.DataFrame(results, index=df_candidates.index)

    df_results['keep'] = df_results['score'] >= ACCEPTANCE_THRESHOLD
    df_final_with_reasons = pd.concat([df_candidates, df_results], axis=1)

    df_accepted = df_final_with_reasons[df_final_with_reasons['keep'] == True].drop(columns=['keep', 'score', 'reason'])
    df_rejected = df_final_with_reasons[df_final_with_reasons['keep'] == False].drop(columns=['keep'])

    print("\n--- 精筛完成 ---")
    df_accepted.to_csv(FINAL_RESULT_FILE, index=False, encoding='utf-8')
    print(f"✅ {len(df_accepted)} 篇最终文章已保存到: {FINAL_RESULT_FILE}")

    df_rejected.to_csv(REJECTED_FILE, index=False, encoding='utf-8')
    print(f"ℹ️ {len(df_rejected)} 篇被拒绝的文章已保存到: {REJECTED_FILE} (供分析)")

    end_time_s2 = time.time()
    print(f"阶段二 (精筛) 耗时: {(end_time_s2 - start_time_s2):.2f} 秒。")

else:
    print("候选集为空或spaCy模型未加载，跳过精筛。")

print("\n✅ 块 5: 精筛流程执行完毕。")

--- 阶段二: 开始精筛候选集 ---
✅ 成功加载 179380 篇候选文章。
开始使用 5 个进程并行处理文本...


精筛文章:   0%|          | 0/179380 [00:00<?, ?it/s]


正在合并精筛结果...

--- 精筛完成 ---
✅ 173841 篇最终文章已保存到: ../data_processed/final_china_news.csv
ℹ️ 5539 篇被拒绝的文章已保存到: ../data_processed/china_news_rejected_articles.csv (供分析)
阶段二 (精筛) 耗时: 1751.41 秒。

✅ 块 5: 精筛流程执行完毕。
